# Regresión lineal - Scikit-learn - Vars. Categóricas

**Explicación R. Lineal ->** https://www.notion.so/Regresi-n-lineal-39b5e113aa01468a8963932b5ec26123

**Librerías, Funciones:**
- *Pandas* -> read_csv.
- *Python* -> .columns.values.tolist(), list(zip()), .drop, .concat, .getDummies, len(), .join()
- *Numpy* -> .mean, .sqrt
- *from sklearn.linear_model import LinearRegression* -> LinearRegression(), .intercept_, .coef_, .score(), fit()

**Índice:**

0. Preparación **previa**
1. Convertimos vars. Categóricas a **Dummies**
2. Construimos el **modelo**
3. Elaboramos una **predicción**
4. Calculamos los **errores**
- Extra. **Enmascarar** vars dummy

# 0. Preparación previa

In [37]:
%config IPCompleter.greedy=True  #Para permitir el autocompletado en Jupyter

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression #Para calcular el lm

In [48]:
mainpath = "/Users/irene/Documents/GitHub/python-ml-course/datasets"  #Ruta ficheros
filename = "ecom-expense/Ecom Expense.csv" #Fichero a abrir
fullpath = mainpath + "/" + filename #Ruta completa

df = pd.read_csv(fullpath)
df.head(3)

#2 vars categóricas -> gender & city tier

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455


# 1. Convertimos vars. Categóricas a Dummies

In [39]:
def createDummies(df, var_name):
    dummy = pd.get_dummies(df[var_name], prefix=var_name)
    df = df.drop(var_name, axis=1)
    df = pd.concat([df, dummy], axis=1)
    return df

df2 = createDummies(df, ["Gender","City Tier"])
df2.head(3)

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Total Spend,Gender_Female,Gender_Male,City Tier_Tier 1,City Tier_Tier 2,City Tier_Tier 3
0,TXN001,42,10,7313,627.668127,5,4198.385084,1,0,1,0,0
1,TXN002,24,8,17747,126.904567,3,4134.976648,1,0,0,1,0
2,TXN003,47,11,22845,873.469701,2,5166.614455,0,1,0,1,0


In [40]:
column_names = df2.columns.values.tolist() #Nombre de las posibles vars predictoras

# 2. Construimos el modelo

**El modelo lineal obtenido por regresión es:**

* Total_Spend = (-79.41713030137362) + (0.14753898049205738 * 'Monthly Income') + ( 0.15494612549589545 * 'Transaction Time') - (131.02501325554567 * 'Gender_Female') + (131.0250132555456 * 'Gender_Male') + (76.76432601049527 * 'City_Tier 1') + (55.138974309232474 * 'City_Tier 2') - (131.9033003197278 * 'City_Tier 3') + (772.2334457445648 * 'Record')

In [41]:
#Establecemos las vars. predictoras
feature_cols = ["Monthly Income", "Transaction Time",
                "Gender_Female", "Gender_Male",
                "City Tier_Tier 1", "City Tier_Tier 2", "City Tier_Tier 3", "Record"]

X = df2[feature_cols]
Y = df2["Total Spend"]


#Creamos el modelo
lm = LinearRegression()
lm.fit(X,Y)

#Obtenemos los parámetros del modelo
print("El valor de alpha es: " + str(lm.intercept_))
print("El valor de R2 es: " + str(lm.score(X,Y)))
print("El valor de las betas es:")
list(zip(feature_cols,lm.coef_)) #Pasamos de lista a tupla:

El valor de alpha es: -79.4171303013718
El valor de R2 es: 0.9179923586131016
El valor de las betas es:


[('Monthly Income', 0.14753898049205738),
 ('Transaction Time', 0.15494612549589634),
 ('Gender_Female', -131.02501325554624),
 ('Gender_Male', 131.02501325554607),
 ('City Tier_Tier 1', 76.76432601049513),
 ('City Tier_Tier 2', 55.1389743092325),
 ('City Tier_Tier 3', -131.9033003197277),
 ('Record', 772.2334457445645)]

# 3. Elaboramos una predicción:

In [42]:
df2["prediction"] = -79.41713030137362 + df2["Monthly Income"]*0.14753898049205738 + df2["Transaction Time"]* 0.15494612549589545+ df2["Gender_Female"] * (-131.02501325554567) + df2["Gender_Male"] * 131.0250132555456+ df2["City Tier_Tier 1"]* 76.76432601049527 +  df2["City Tier_Tier 2"]* 55.138974309232474 + df2["City Tier_Tier 3"]* (-131.9033003197278)+ df2["Record"]* 772.2334457445648

# 4. Calculamos los errores:

In [43]:
SSD = sum((df2["prediction"]-df["Total Spend"])**2)
RSE = np.sqrt(SSD/(len(df2)-len(feature_cols)-1))
sales_mean = np.mean(df2["Total Spend"])
error = (RSE / sales_mean)*100

print("El SSD es: " + str(SSD))
print("El RSE es: " + str(RSE))
print("La media de ventas es: " + str(sales_mean))
print("El Error es: " + str(error) + "%") #13% not bad

El SSD es: 1517733985.3408136
El RSE es: 803.1318809818158
La media de ventas es: 6163.176415976714
El Error es: 13.03113568029415%


# Extra. Enmascarar vars dummy

Si una persona no es Female, es Male. Lo mismo pasa con los Tiers. 
Por ende, hay vars. dummy de entrada **redundantes** -> se puede "*enmascarar*" una dummy para reducir a la mitad las vars. del modelo, **sin modificarlo:**
    
**iloc[:,1:] #Todas las filas, de la columna 1 en adelante**
    

In [44]:
dummy_gender = pd.get_dummies(df["Gender"], prefix="Gender").iloc[:,1:]
dummy_gender.head(3)

,Gender_Male
0,0
1,0
2,1


In [45]:
dummy_city_tier = pd.get_dummies(df["City Tier"], prefix="City").iloc[:,1:]
dummy_city_tier.head(3)

,City_Tier 2,City_Tier 3
0,0,0
1,1,0
2,1,0


In [46]:
column_names = df.columns.values.tolist() #Columnas iniciales
df3 = df[column_names].join(dummy_gender).join(dummy_city_tier) #Columnas iniciales + las nuevas dummies
df3.head(3)

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Male,City_Tier 2,City_Tier 3
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,0,0,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,0,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,1,1,0


In [47]:
#Establecemos las vars. predictoras
feature_cols3 = ["Monthly Income", "Transaction Time",
                "Gender_Male",
                "City_Tier 2", "City_Tier 3", "Record"]

X = df3[feature_cols3]
Y = df3["Total Spend"]

#Creamos el modelo
lm3 = LinearRegression()
lm3.fit(X,Y)

#Obtenemos los parámetros del modelo
print("El valor de alpha es: " + str(lm3.intercept_))
print("El valor de R2 es: " + str(lm3.score(X,Y))) #Es el mismo modelo, R2 es igual.
print("El valor de las betas es:")
list(zip(feature_cols,lm3.coef_)) #Pasamos de lista a tupla:

El valor de alpha es: -133.67781754642238
El valor de R2 es: 0.9179923586131016
El valor de las betas es:


[('Monthly Income', 0.14753898049205744),
 ('Transaction Time', 0.1549461254959002),
 ('Gender_Female', 262.0500265110948),
 ('Gender_Male', -21.62535170126276),
 ('City Tier_Tier 1', -208.66762633022296),
 ('City Tier_Tier 2', 772.2334457445636)]

### Explicación:

Es el mismo modelo, únicamente aglutinamos la info:

Coeficientes con todas las variables en el modelo anterior:
* ('Monthly Income', 0.14753898049205738),
* ('Transaction Time', 0.15494612549589545),
* ('Gender_Female', -131.02501325554567),
* ('Gender_Male', 131.0250132555456),
* ('City_Tier 1', 76.76432601049527),
* ('City_Tier 2', 55.138974309232474),
* ('City_Tier 3', -131.9033003197278),
* ('Record', 772.2334457445648)
 
Coeficientes tras enmascarar las variables dummy pertinentes
* 'Monthly Income', 0.14753898049205744),
* ('Transaction Time', 0.15494612549589631),
* ('Gender_Male', 262.05002651109595),
* ('City_Tier 2', -21.62535170126296),
* ('City_Tier 3', -208.66762633022324),
* ('Record', 772.2334457445635)]

Los cambios se reflejan en
* Gender_Male: 
    * antes -> 131.02, 
    * después -> 262.05 = ( 131.02 - (-131.02)) #Hombre + mujer
* Gender_Female: 
    * antes -> -131.02,
    * después -> 0
* CT1: 
    * antes -> 76.76,
    * después -> 0
* CT2: 
    * antes -> 55.13, 
    * después -> -21.62 = (55.13 - 76.76) #CT2 + CT1
* CT3: 
    * antes -> -131.90, 
    * después -> -208.66 = (-131.90 - 76.76) #CT3 + CT1
    